<a href="https://colab.research.google.com/github/deRockerTom/dataFrenchWords/blob/main/compressFrenchFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/deRockerTom/dataFrenchWords
!pip install textract
!sudo apt-get install antiword

Cloning into 'dataFrenchWords'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 113 (delta 19), reused 0 (delta 0), pack-reused 78
Receiving objects: 100% (113/113), 6.21 MiB | 27.75 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 11.2 MB/s 
     |████████████████████████████████| 103 kB 44.5 MB/s 
     |████████████████████████████████| 5.6 MB 37.8 MB/s 
     |████████████████████████████████| 106 kB 68.9 MB/s 
     |████████████████████████████████| 32.8 MB 1.2 MB/s 
     |████████████████████████████████| 69 kB 8.7 MB/s 
     |████████████████████████████████| 2.3 MB 56.0 MB/s 
     |████████████████████████████████| 128 kB 63.6 MB/s 
     |████████████████████████████████| 112 kB 73.0 MB/s 
     |████████████████████████████████| 73 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  antiword
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 128 kB of archives.
After this operation, 633 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 antiword amd64 0.37-11build1 [128 kB]
Fetched 128 kB in 0s (309 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-pre

In [2]:
import textract
text = textract.process("dataFrenchWords/maupassant_bel_ami_illustre.doc")
text = text.decode("utf-8")
text = text.replace('’', '\'').replace('–', '-').replace('…', '...').replace('‘', '\'').replace('€', 'euros').replace('[pic]', '').replace("“", "\"").replace("”", "\"").replace("—", "-")

In [3]:
# Get dict with file
import os

def getDictWithFile(file):
  with open(file, 'r') as f:
    dic = {}
    key = ""
    value = ""
    isKey = True
    for l in f.readlines():
      for c in l:
        if c == 'ក':
          isKey = False
        elif c == 'ខ':
          isKey = True
          dic[key] = int(value)
          key = ""
          value = ""
        else:
          if isKey:
            key = key + c
          else:
            value = value + c
    value = value + "\n"
  return dic

dicByHugo = getDictWithFile('dataFrenchWords/dicByHugo')
dicByHugoMax = getDictWithFile('dataFrenchWords/dicByHugoMax')
dictByHugoMax10 = getDictWithFile('dataFrenchWords/dicByHugoMax10')
dictByHugoMax50 = getDictWithFile('dataFrenchWords/dicByHugoMax50')




In [4]:
def convertToBinary(number):
  binary = ''
  while number > 0:
    binary = str(number % 2) + binary
    number = number // 2
  return binary
def convertToNineBits(bits):
  n = len(bits)
  if n < 9:
    k = 9 - n
    return str('0') * k + bits
  elif n == 9:
    return bits
  else:
    print("Attention : un des encodages est trop grand (>9)")
    print(bits)

convertToNineBits(convertToBinary(255))

'011111111'

In [5]:
import csv

# open csv
dic = {}
with open('dataFrenchWords/french-word-list-verbs.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-adjectives.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-nouns.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

# Ponctuation :

dic[', '] = 150000
dic['. '] = 150000

dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True))
dic = {k: dic[k] for k in list(dic)[:256]} # In order to code the dictionnary on only 2 other bytes with most frequent words
offset = 256
i = 0
for key in dic:
  dic[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1
dic2 = {}
offset = 256
for key in dicByHugo:
  dic2[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1
dic3 = {}
offset = 256
for key in dicByHugoMax:
  dic3[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1
dic4 = {}
offset = 256
for key in dictByHugoMax10:
  dic4[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1
dic5 = {}
offset = 256
for key in dictByHugoMax50:
  dic5[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1

print(dic5)

reverted_dic = {v: k for k, v in dic.items()}
reverted_dic2 = {v: k for k, v in dic2.items()}
reverted_dic3 = {v: k for k, v in dic3.items()}
reverted_dic4 = {v: k for k, v in dic4.items()}
reverted_dic5 = {v: k for k, v in dic5.items()}

print(reverted_dic5)

{'  ': '100000000', 'e ': '100000001', 's ': '100000010', 't ': '100000011', ' d': '100000100', ', ': '100000101', 'ai': '100000110', 'es': '100000111', ' l': '100001000', 'le': '100001001', 'en': '100001010', 're': '100001011', 'ou': '100001100', 'de': '100001101', ' s': '100001110', 'nt': '100001111', 'it': '100010000', ' p': '100010001', 'on': '100010010', ' c': '100010011', 'an': '100010100', 'n ': '100010101', ' e': '100010110', 'ur': '100010111', 'te': '100011000', 'er': '100011001', 'is': '100011010', 'r ': '100011011', '\n ': '100011100', ' a': '100011101', '\n\n': '100011110', 'a ': '100011111', 'qu': '100100000', '   ': '100100001', 'se': '100100010', 'ne': '100100011', 'et': '100100100', ' m': '100100101', 'me': '100100110', 'la': '100100111', ' de': '100101000', 'ie': '100101001', 'il': '100101010', 'e  ': '100101011', 'eu': '100101100', ' t': '100101101', 'ra': '100101110', 'ar': '100101111', 'in': '100110000', 'll': '100110001', 'ui': '100110010', 'l ': '100110011', 'es '

In [6]:
# class to build a tree defining all the possible  string in dic
class SuffixTree():
  class Node():
    def __init__(self,char):
      self.char = char
      self.out = {}
      self.isFinal = False
    def __str__(self, level=0):
      ret = "  " * level + str(self.char) + "\n"
      for c in self.out:
        ret += self.out[c].__str__(level + 1)
      return ret

  def __init__(self, dic):
    self.root = self.Node(None)
    self.root.isFinal = True
    for key in dic:
      cursor = self.root
      for c in key:
        if c in cursor.out:
          cursor = cursor.out[c]
        else:
          node = self.Node(c)
          cursor.out[c] = node
          cursor = node
      cursor.isFinal = True
  def longestPrefix(self, s):
    cursor = self.root
    word = ''
    temp = ''
    end = False
    i = 0
    length = len(s)
    while not(end) and i < length:
      if s[i] in cursor.out:
        temp = temp + s[i]
        cursor = cursor.out[s[i]]
        if cursor.isFinal:
          word = temp
        i += 1
      else:
        end = True
    return word
  def __str__(self):
    return f'{self.root}'

s = SuffixTree({'ab' : 1, 'abc': 2, 'coucou' : 5, 'aba' : 4, 'aaa' : 8})
print(s)
s.longestPrefix('abdc')
    


None
  a
    b
      c
      a
    a
      a
  c
    o
      u
        c
          o
            u



'ab'

In [7]:
tree = SuffixTree(dic)
tree2 = SuffixTree(dic2)
tree3 = SuffixTree(dic3)
tree4 = SuffixTree(dic4)
tree5 = SuffixTree(dic5)

In [8]:
def compress(string, tree, dic):
  i = 0
  ret = ''
  while i<len(string):
    temp = tree.longestPrefix(string[i:])
    n = len(temp)
    if n == 0:
      bit = convertToNineBits(convertToBinary(ord(string[i])))
      ret += bit
      i += 1
    else:
      bit = dic[temp]
      ret += bit
      i += n
  return ret



def decompress(string, dic):
  ret = ''
  for i in range(0, len(string) - 1, 9):
    if int(string[i : i + 9], 2) > 255:
      ret += dic[string[i : i + 9]]
    else:
      ret += chr(int(string[i : i + 9], 2))
  return ret

In [9]:
# Multi process version

import multiprocessing
from multiprocessing import Pool
from functools import partial

def compressChunk(string, tree, dic):
  i = 0
  ret = ''
  while i<len(string):
    temp = tree.longestPrefix(string[i:])
    n = len(temp)
    if n == 0:
      bit = convertToNineBits(convertToBinary(ord(string[i])))
      ret += bit
      i += 1
    else:
      bit = dic[temp]
      ret += bit
      i += n
  return ret

def compressMulti(string, tree, dic):
  chunks, chunk_size = len(string), len(string)//os.cpu_count()
  string_list = [ string[i:i+chunk_size] for i in range(0, chunks, chunk_size) ]
  with Pool() as p:
    ret_list = p.map(partial(compressChunk, tree=tree, dic=dic), string_list)
  ret = ''.join(ret_list)
  return ret

def decompressChunk(string, dic):
  ret = ''
  for i in range(0, len(string) - 1, 9):
    if int(string[i : i + 9], 2) > 255:
      ret += dic[string[i : i + 9]]
    else:
      ret += chr(int(string[i : i + 9], 2))
  return ret

def decompressMulti(string, dic):
  chunks, chunk_size = len(string), (len(string)//(os.cpu_count() * 9)) * 9
  string_list = [ string[i:i+chunk_size] for i in range(0, chunks, chunk_size) ]
  with Pool() as p:
    ret_list = p.map(partial(decompressChunk, dic=dic), string_list)
  ret = ''.join(ret_list)
  return ret

In [10]:
# Next step on compression: Huffman

from re import I
import heapq
from heapq import heappop, heappush, heapify

class Huffman():
  def __init__(self, ch, freq, left=None, right=None):
    self.ch = ch
    self.freq = freq
    self.left = left
    self.right = right

  def __lt__(self, other):
    return self.freq < other.freq

  def __str__(self):
    return f'{self.ch}: {self.freq}'


def print2DTree(root, space=0, LEVEL_SPACE=12):
  if (root == None):
    return
  space += LEVEL_SPACE
  print2DTree(root.right, space)
  # print() # neighbor space
  for i in range(LEVEL_SPACE, space):
    print(end=" ")
  print("|" + str(root) +
      ("|<" if not(root.left is None and root.right is None) else ""))
  print2DTree(root.left, space)


def isLeaf(root):
  return root.left is None and root.right is None


def encode(root, s, code):
  if root is None:
    return
  if isLeaf(root):
    code[root.ch] = s if len(s) > 0 else '1'
  encode(root.left, s + '0', code)
  encode(root.right, s + '1', code)


def encodeString(string, code):
  s = ''
  for i in range(0, len(string), 9):
    s += code[string[i:i+9]]
  return s


def decode(root, i, s):
  if root is None:
    return i, ''
  if isLeaf(root):
    # print(root.ch, end='')
    return i, root.ch
  i += 1
  root = root.left if s[i] == '0' else root.right
  return decode(root, i, s)


def getFreq(text):
  s = set()
  for i in range(0, len(text), 9):
    s.add(text[i:i+9])
  return {i: text.count(i) for i in s}


def buildHuffmanTree(text, print_tree=False):
  if len(text) == 0:
    return
  freq = getFreq(text)
  pq = [Huffman(ch, freq[ch]) for ch in freq]
  # pq.sort(reverse=True)
  heapify(pq)
  while len(pq) > 1:
    left = heappop(pq)
    right = heappop(pq)
    total = left.freq + right.freq
    heappush(pq, Huffman(None, total, left, right))

  root = pq[0]
  code = {}
  encode(root, '', code)
  if print_tree:
    print2DTree(root)
  return code, root

def decodeString(string, hufTree):
  decoded_string = ''
  if isLeaf(hufTree):
    decoded_string = hufTree.ch * len(string)
  else:
    i = -1
    while i < len(string) - 1:
      i, ch = decode(hufTree, i, string)
      decoded_string += ch
  return decoded_string



In [55]:
def get_binary(s):
  while len(s)%8 != 0:
    s += '0'
  return bytearray(int(s[x:x+8], 2) for x in range(0, len(s), 8))

In [38]:
!mkdir mycompressed

#**Tests sur la base d'apprentissage du dictionnaire**

In [52]:
import time

In [12]:
# First dictionnary not based on a book
t1_1s = time.time()
compressed = compress(text, tree, dic)

# decompressed = decompress(compressed, reverted_dic)

code, hufTree = buildHuffmanTree(compressed)

compressed_2 = encodeString(compressed, code)
t1_1e = time.time()

t1_2s = time.time()
compressed_parallel = compressMulti(text, tree, dic)

code_parallel, hufTree_parallel = buildHuffmanTree(compressed_parallel)

compressed_parallel_2 = encodeString(compressed_parallel, code_parallel)
t1_2e = time.time()


print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed)} bits puis {len(compressed_2)} après Huffman')
print(f'Le taux de compression est donc de {len(compressed) / (len(text) * 8)} puis de {len(compressed_2) / (len(text) * 8)} après Huffman')

print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed_parallel)} bits puis {len(compressed_parallel_2)} après Huffman pour la version parallele')
print(f'Le taux de compression est donc de {len(compressed_parallel) / (len(text) * 8)} puis de {len(compressed_parallel_2) / (len(text) * 8)} après Huffman parallele')

print(f'La compression basique a mis {round(t1_1e - t1_1s, 2)} secondes et la compression en parallele a mis {round(t1_2e - t1_2s, 2)} secondes')

Le fichier avant décompression faisait 5442960 bits et celui compressé fait 5360544 bits puis 4023128 après Huffman
Le taux de compression est donc de 0.9848582388994224 puis de 0.7391434072636948 après Huffman
Le fichier avant décompression faisait 5442960 bits et celui compressé fait 5360544 bits puis 4023128 après Huffman pour la version parallele
Le taux de compression est donc de 0.9848582388994224 puis de 0.7391434072636948 après Huffman parallele
La compression basique a mis 68.98 secondes et la compression en parallele a mis 28.08 secondes


In [13]:
# Second dictionnary (made with only strings of 3 char)
t2_1s = time.time()
compressed2 = compress(text, tree2, dic2)

# decompressed2 = decompress(compressed2, reverted_dic2)

code2, hufTree2 = buildHuffmanTree(compressed2)

compressed2_2 = encodeString(compressed2, code2)
t2_1e = time.time()

t2_2s = time.time()
compressed_parallel2 = compressMulti(text, tree2, dic2)

code_parallel2, hufTree_parallel2 = buildHuffmanTree(compressed_parallel2)

compressed_parallel2_2 = encodeString(compressed_parallel2, code_parallel2)
t2_2e = time.time()


print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed2)} bits puis {len(compressed2_2)} après Huffman')
print(f'Le taux de compression est donc de {len(compressed2) / (len(text) * 8)} puis de {len(compressed2_2) / (len(text) * 8)} après Huffman')

print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed_parallel2)} bits puis {len(compressed_parallel2_2)} après Huffman pour la version parallele')
print(f'Le taux de compression est donc de {len(compressed_parallel2) / (len(text) * 8)} puis de {len(compressed_parallel2_2) / (len(text) * 8)} après Huffman parallele')

print(f'La compression basique a mis {round(t2_1e - t2_1s, 2)} secondes et la compression en parallele a mis {round(t2_2e - t2_2s, 2)} secondes')

Le fichier avant décompression faisait 5442960 bits et celui compressé fait 3422448 bits puis 3056526 après Huffman
Le taux de compression est donc de 0.6287843379337713 puis de 0.56155584461396 après Huffman
Le fichier avant décompression faisait 5442960 bits et celui compressé fait 3422448 bits puis 3056526 après Huffman pour la version parallele
Le taux de compression est donc de 0.6287843379337713 puis de 0.56155584461396 après Huffman parallele
La compression basique a mis 44.48 secondes et la compression en parallele a mis 20.48 secondes


In [14]:
# Third dictionnary (made with strings of 2-3 char)
t3_1s = time.time()
compressed3 = compress(text, tree3, dic3)

# decompressed3 = decompress(compressed3, reverted_dic3)

code3, hufTree3 = buildHuffmanTree(compressed3)

compressed3_2 = encodeString(compressed3, code3)
t3_1e = time.time()

t3_2s = time.time()
compressed_parallel3 = compressMulti(text, tree3, dic3)

code_parallel3, hufTree_parallel3 = buildHuffmanTree(compressed_parallel3)

compressed_parallel3_2 = encodeString(compressed_parallel3, code_parallel3)
t3_2e = time.time()

print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed3)} bits puis {len(compressed3_2)} après Huffman')
print(f'Le taux de compression est donc de {len(compressed3) / (len(text) * 8)} puis de {len(compressed3_2) / (len(text) * 8)} après Huffman')

print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed_parallel3)} bits puis {len(compressed_parallel3_2)} après Huffman pour la version parallele')
print(f'Le taux de compression est donc de {len(compressed_parallel3) / (len(text) * 8)} puis de {len(compressed_parallel3_2) / (len(text) * 8)} après Huffman parallele')

print(f'La compression basique a mis {round(t3_1e - t3_1s, 2)} secondes et la compression en parallele a mis {round(t3_2e - t3_2s, 2)} secondes')

Le fichier avant décompression faisait 5442960 bits et celui compressé fait 3052683 bits puis 2835925 après Huffman
Le taux de compression est donc de 0.5608497949645046 puis de 0.5210262430736217 après Huffman
Le fichier avant décompression faisait 5442960 bits et celui compressé fait 3052683 bits puis 2835925 après Huffman pour la version parallele
Le taux de compression est donc de 0.5608497949645046 puis de 0.5210262430736217 après Huffman parallele
La compression basique a mis 39.63 secondes et la compression en parallele a mis 18.67 secondes


In [15]:
# Fourth dictionnary (made with only strings of 2-9 char)
t4_1s = time.time()
compressed4 = compress(text, tree4, dic4)

# decompressed4 = decompress(compressed4, reverted_dic4)

code4, hufTree4 = buildHuffmanTree(compressed4)

compressed4_2 = encodeString(compressed4, code4)

t4_1e = time.time()

t4_2s = time.time()
compressed_parallel4 = compressMulti(text, tree4, dic4)

code_parallel4, hufTree_parallel4 = buildHuffmanTree(compressed_parallel4)

compressed_parallel4_2 = encodeString(compressed_parallel4, code_parallel4)
t4_2e = time.time()

print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed4)} bits puis {len(compressed4_2)} après Huffman')
print(f'Le taux de compression est donc de {len(compressed4) / (len(text) * 8)} puis de {len(compressed4_2) / (len(text) * 8)} après Huffman')

print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed_parallel4)} bits puis {len(compressed_parallel4_2)} après Huffman pour la version parallele')
print(f'Le taux de compression est donc de {len(compressed_parallel4) / (len(text) * 8)} puis de {len(compressed_parallel4_2) / (len(text) * 8)} après Huffman parallele')

print(f'La compression basique a mis {round(t4_1e - t4_1s, 2)} secondes et la compression en parallele a mis {round(t4_2e - t4_2s, 2)} secondes')

Le fichier avant décompression faisait 5442960 bits et celui compressé fait 3031308 bits puis 2806662 après Huffman
Le taux de compression est donc de 0.5569227038229199 puis de 0.5156499404735658 après Huffman
Le fichier avant décompression faisait 5442960 bits et celui compressé fait 3031308 bits puis 2806662 après Huffman pour la version parallele
Le taux de compression est donc de 0.5569227038229199 puis de 0.5156499404735658 après Huffman parallele
La compression basique a mis 39.12 secondes et la compression en parallele a mis 18.5 secondes


In [16]:
# Fifth dictionnary (made with only strings of 2-49 char)
t5_1s = time.time()
compressed5 = compress(text, tree5, dic5)

# decompressed5 = decompress(compressed5, reverted_dic5)

code5, hufTree5 = buildHuffmanTree(compressed5)

compressed5_2 = encodeString(compressed5, code5)
t5_1e = time.time()

t5_2s = time.time()
compressed_parallel5 = compressMulti(text, tree5, dic5)

code_parallel5, hufTree_parallel5 = buildHuffmanTree(compressed_parallel5)

compressed_parallel5_2 = encodeString(compressed_parallel5, code_parallel5)
t5_2e = time.time()

print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed5)} bits puis {len(compressed5_2)} après Huffman')
print(f'Le taux de compression est donc de {len(compressed5) / (len(text) * 8)} puis de {len(compressed5_2) / (len(text) * 8)} après Huffman')

print(f'Le fichier avant décompression faisait {len(text) * 8} bits et celui compressé fait {len(compressed_parallel5)} bits puis {len(compressed_parallel5_2)} après Huffman pour la version parallele')
print(f'Le taux de compression est donc de {len(compressed_parallel5) / (len(text) * 8)} puis de {len(compressed_parallel5_2) / (len(text) * 8)} après Huffman parallele')

print(f'La compression basique a mis {round(t5_1e - t5_1s, 2)} secondes et la compression en parallele a mis {round(t5_2e - t5_2s, 2)} secondes')

Le fichier avant décompression faisait 5442960 bits et celui compressé fait 3031308 bits puis 2806662 après Huffman
Le taux de compression est donc de 0.5569227038229199 puis de 0.5156499404735658 après Huffman
Le fichier avant décompression faisait 5442960 bits et celui compressé fait 3031308 bits puis 2806662 après Huffman pour la version parallele
Le taux de compression est donc de 0.5569227038229199 puis de 0.5156499404735658 après Huffman parallele
La compression basique a mis 39.02 secondes et la compression en parallele a mis 18.48 secondes


In [56]:
with open('mycompressed/maupassant_bel_ami_illustre.cpsd', 'wb') as f:
  f.write(get_binary(compressed_parallel5_2))

#**Test du dictionnaire sur un autre livre, ici le premier tome du livre "Les misérables"**

In [17]:
import textract
text2 = textract.process("dataFrenchWords/hugo_les_miserables_fantine_source.doc")
text2 = text2.decode("utf-8")
text2 = text2.replace('’', '\'').replace('–', '-').replace('…', '...').replace('‘', '\'').replace('€', 'euros').replace('[pic]', '').replace("“", "\"").replace("”", "\"").replace("—", "-")

In [18]:
t6_1s = time.time()
compressed6 = compress(text2, tree5, dic5)

# decompressed6 = decompress(compressed6, reverted_dic5)

code6, hufTree6 = buildHuffmanTree(compressed6)

compressed6_2 = encodeString(compressed6, code6)
t6_1e = time.time()

t6_2s = time.time()
compressed_parallel6 = compressMulti(text2, tree5, dic5)

code_parallel6, hufTree_parallel6 = buildHuffmanTree(compressed_parallel6)

compressed_parallel6_2 = encodeString(compressed_parallel6, code_parallel6)
t6_2e = time.time()

print(f'Le fichier avant décompression faisait {len(text2) * 8} bits et celui compressé fait {len(compressed6)} bits puis {len(compressed6_2)} après Huffman')
print(f'Le taux de compression est donc de {len(compressed6) / (len(text2) * 8)} puis de {len(compressed6_2) / (len(text2) * 8)} après Huffman')

print(f'Le fichier avant décompression faisait {len(text2) * 8} bits et celui compressé fait {len(compressed_parallel6)} bits puis {len(compressed_parallel6_2)} après Huffman pour la version parallele')
print(f'Le taux de compression est donc de {len(compressed_parallel6) / (len(text2) * 8)} puis de {len(compressed_parallel6_2) / (len(text2) * 8)} après Huffman parallele')

print(f'La compression basique a mis {round(t6_1e - t6_1s, 2)} secondes et la compression en parallele a mis {round(t6_2e - t6_2s, 2)} secondes')

Le fichier avant décompression faisait 6131592 bits et celui compressé fait 3502737 bits puis 3252788 après Huffman
Le taux de compression est donc de 0.5712606122520872 puis de 0.5304964844366683 après Huffman
Le fichier avant décompression faisait 6131592 bits et celui compressé fait 3502737 bits puis 3252788 après Huffman pour la version parallele
Le taux de compression est donc de 0.5712606122520872 puis de 0.5304964844366683 après Huffman parallele
La compression basique a mis 49.83 secondes et la compression en parallele a mis 22.79 secondes


In [42]:
with open('mycompressed/hugo_les_miserables_fantine_source.cpsd', 'w') as f:
  f.write(compressed_parallel6_2)

#**Test avec un Roman de Science-fiction**

In [19]:
import textract
text3 = textract.process("dataFrenchWords/abbot_flatland_source.doc")
text3 = text3.decode("utf-8")
text3 = text3.replace('’', '\'').replace('–', '-').replace('…', '...').replace('‘', '\'').replace('€', 'euros').replace('[pic]', '').replace("“", "\"").replace("”", "\"").replace("—", "-")

In [20]:
t7_1s = time.time()
compressed7 = compress(text3, tree5, dic5)

# decompressed7 = decompress(compressed7, reverted_dic5)

code7, hufTree7 = buildHuffmanTree(compressed7)

compressed7_2 = encodeString(compressed7, code7)
t7_1e = time.time()

t7_2s = time.time()
compressed_parallel7 = compressMulti(text3, tree5, dic5)

code_parallel7, hufTree_parallel7 = buildHuffmanTree(compressed_parallel7)

compressed_parallel7_2 = encodeString(compressed_parallel7, code_parallel7)
t7_2e = time.time()

print(f'Le fichier avant décompression faisait {len(text3) * 8} bits et celui compressé fait {len(compressed7)} bits puis {len(compressed7_2)} après Huffman')
print(f'Le taux de compression est donc de {len(compressed7) / (len(text3) * 8)} puis de {len(compressed7_2) / (len(text3) * 8)} après Huffman')

print(f'Le fichier avant décompression faisait {len(text3) * 8} bits et celui compressé fait {len(compressed_parallel7)} bits puis {len(compressed_parallel7_2)} après Huffman pour la version parallele')
print(f'Le taux de compression est donc de {len(compressed_parallel7) / (len(text3) * 8)} puis de {len(compressed_parallel7_2) / (len(text) * 8)} après Huffman parallele')

print(f'La compression basique a mis {round(t7_1e - t7_1s, 2)} secondes et la compression en parallele a mis {round(t7_2e - t7_2s, 2)} secondes')

Le fichier avant décompression faisait 1955288 bits et celui compressé fait 1122732 bits puis 1035390 après Huffman
Le taux de compression est donc de 0.5742028795757965 puis de 0.5295332452303702 après Huffman
Le fichier avant décompression faisait 1955288 bits et celui compressé fait 1122741 bits puis 1035445 après Huffman pour la version parallele
Le taux de compression est donc de 0.5742074824782846 puis de 0.19023564384085132 après Huffman parallele
La compression basique a mis 6.57 secondes et la compression en parallele a mis 3.99 secondes


In [58]:
with open('mycompressed/abbot_flatland_source.cpsd', 'wb') as f:
  f.write(get_binary(compressed_parallel7_2))

#**Test avec des Nouvelles - Contes Humour**

In [21]:
import textract
text4 = textract.process("dataFrenchWords/allais_a_se_tordre_source.doc")
text4 = text4.decode("utf-8")
text4 = text4.replace('’', '\'').replace('–', '-').replace('…', '...').replace('‘', '\'').replace('€', 'euros').replace('[pic]', '').replace("“", "\"").replace("”", "\"").replace("—", "-")

In [22]:
t8_1s = time.time()
compressed8 = compress(text4, tree5, dic5)

# decompressed8 = decompress(compressed8, reverted_dic5)

code8, hufTree8 = buildHuffmanTree(compressed8)

compressed8_2 = encodeString(compressed8, code8)
t8_1e = time.time()

t8_2s = time.time()
compressed_parallel8 = compressMulti(text4, tree5, dic5)

code_parallel8, hufTree_parallel8 = buildHuffmanTree(compressed_parallel8)

compressed_parallel8_2 = encodeString(compressed_parallel8, code_parallel8)
t8_2e = time.time()

print(f'Le fichier avant décompression faisait {len(text4) * 8} bits et celui compressé fait {len(compressed8)} bits puis {len(compressed8_2)} après Huffman')
print(f'Le taux de compression est donc de {len(compressed8) / (len(text4) * 8)} puis de {len(compressed8_2) / (len(text4) * 8)} après Huffman')

print(f'Le fichier avant décompression faisait {len(text4) * 8} bits et celui compressé fait {len(compressed_parallel8)} bits puis {len(compressed_parallel8_2)} après Huffman pour la version parallele')
print(f'Le taux de compression est donc de {len(compressed_parallel8) / (len(text4) * 8)} puis de {len(compressed_parallel8_2) / (len(text4) * 8)} après Huffman parallele')

print(f'La compression basique a mis {round(t8_1e - t8_1s, 2)} secondes et la compression en parallele a mis {round(t8_2e - t8_2s, 2)} secondes')

Le fichier avant décompression faisait 1935936 bits et celui compressé fait 1101645 bits puis 1015447 après Huffman
Le taux de compression est donc de 0.5690503198452841 puis de 0.5245250876062019 après Huffman
Le fichier avant décompression faisait 1935936 bits et celui compressé fait 1101654 bits puis 1015458 après Huffman pour la version parallele
Le taux de compression est donc de 0.5690549687592978 puis de 0.5245307696122186 après Huffman parallele
La compression basique a mis 6.31 secondes et la compression en parallele a mis 3.9 secondes


In [59]:
with open('mycompressed/allais_a_se_tordre_source.cpsd', 'wb') as f:
  f.write(get_binary(compressed_parallel8_2))

#**Test avec des Nouvelles - Contes Jeunesse**

In [23]:
import textract
text5 = textract.process("dataFrenchWords/andersen_contes_tome1_source.doc")
text5 = text5.decode("utf-8")
text5 = text5.replace('’', '\'').replace('–', '-').replace('…', '...').replace('‘', '\'').replace('€', 'euros').replace('[pic]', '').replace("“", "\"").replace("”", "\"").replace("—", "-")

In [24]:
t9_1s = time.time()
compressed9 = compress(text5, tree5, dic5)

decompressed9 = decompress(compressed9, reverted_dic5)

code9, hufTree9 = buildHuffmanTree(compressed9)

compressed9_2 = encodeString(compressed9, code9)
t9_1e = time.time()

t9_2s = time.time()
compressed_parallel9 = compressMulti(text5, tree5, dic5)

code_parallel9, hufTree_parallel9 = buildHuffmanTree(compressed_parallel9)

compressed_parallel9_2 = encodeString(compressed_parallel9, code_parallel9)
t9_2e = time.time()

print(f'Le fichier avant décompression faisait {len(text5) * 8} bits et celui compressé fait {len(compressed9)} bits puis {len(compressed9_2)} après Huffman')
print(f'Le taux de compression est donc de {len(compressed9) / (len(text5) * 8)} puis de {len(compressed9_2) / (len(text5) * 8)} après Huffman')

print(f'Le fichier avant décompression faisait {len(text5) * 8} bits et celui compressé fait {len(compressed_parallel9)} bits puis {len(compressed_parallel9_2)} après Huffman pour la version parallele')
print(f'Le taux de compression est donc de {len(compressed_parallel9) / (len(text5) * 8)} puis de {len(compressed_parallel9_2) / (len(text5) * 8)} après Huffman parallele')

print(f'La compression basique a mis {round(t9_1e - t9_1s, 2)} secondes et la compression en parallele a mis {round(t9_2e - t9_2s, 2)} secondes')

Le fichier avant décompression faisait 3378528 bits et celui compressé fait 1883835 bits puis 1746725 après Huffman
Le taux de compression est donc de 0.5575904654334669 puis de 0.51700770276286 après Huffman
Le fichier avant décompression faisait 3378528 bits et celui compressé fait 1883835 bits puis 1746725 après Huffman pour la version parallele
Le taux de compression est donc de 0.5575904654334669 puis de 0.51700770276286 après Huffman parallele
La compression basique a mis 16.12 secondes et la compression en parallele a mis 8.51 secondes


In [62]:
with open('mycompressed/anderson_contes_tome1_source.cpsd', 'wb') as f:
  f.write(get_binary(compressed_parallel9_2))

In [25]:
import os, platform, subprocess, re

def get_processor_name():
    if platform.system() == "Windows":
        return platform.processor()
    elif platform.system() == "Darwin":
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin'
        command ="sysctl -n machdep.cpu.brand_string"
        return subprocess.check_output(command).strip()
    elif platform.system() == "Linux":
        command = "cat /proc/cpuinfo"
        all_info = subprocess.check_output(command, shell=True).decode().strip()
        for line in all_info.split("\n"):
            if "model name" in line:
                return re.sub( ".*model name.*:", "", line,1)
    return ""

get_processor_name()

' Intel(R) Xeon(R) CPU @ 2.20GHz'

In [28]:
!mkdir notcompressed

In [27]:
!mkdir compressed

In [30]:
with open('notcompressed/abbot_flatland_source.txt', 'w') as f:
  f.write(text3)

with open('notcompressed/allais_a_se_tordre_source.txt', 'w') as f:
  f.write(text4)

with open('notcompressed/anderson_contes_tome1_source.txt', 'w') as f:
  f.write(text5)

with open('notcompressed/hugo_les_miserables_fantine_source.txt', 'w') as f:
  f.write(text2)

with open('notcompressed/bel_ami_illustre.txt', 'w') as f:
  f.write(text)

In [35]:
tg1_s = time.time()
!gzip -c notcompressed/abbot_flatland_source.txt > compressed/abbot_flatland_source.gz
tg1_e = time.time()

In [68]:
print(f'La compression de abbot_flatland_source a mis {round(tg1_e - tg1_s, 2)} secondes avec gzip et {round(t7_2e - t7_2s, 2)} avec l\'algorithme personnalisé')
print(f'La compression avec gzip prend {os.path.getsize("compressed/abbot_flatland_source.gz")} bytes et celle de l\'algorithme personnalisé prend {os.path.getsize("mycompressed/abbot_flatland_source.cpsd")} bytes avec une taille initiale de {os.path.getsize("notcompressed/abbot_flatland_source.txt")} bytes')

La compression de abbot_flatland_source a mis 0.12 secondes avec gzip et 3.99 avec l'algorithme personnalisé
La compression avec gzip prend 94329 bytes et celle de l'algorithme personnalisé prend 129431 bytes avec une taille initiale de 253056 bytes


In [63]:
tg2_s = time.time()
!gzip -c notcompressed/allais_a_se_tordre_source.txt > compressed/allais_a_se_tordre_source.gz
tg2_e = time.time()

In [64]:
tg3_s = time.time()
!gzip -c notcompressed/anderson_contes_tome1_source.txt > compressed/andersen_contes_tome1_source.gz
tg3_e = time.time()

In [65]:
tg4_s = time.time()
!gzip -c notcompressed/bel_ami_illustre.txt > compressed/bel_ami_illustre.gz
tg4_e = time.time()

In [66]:
tg5_s = time.time()
!gzip -c notcompressed/hugo_les_miserables_fantine_source.txt > compressed/hugo_les_miserables_fantine_source.gz
tg5_e = time.time()